In [1]:
# load model to predict
from sklearn.externals import joblib
model = joblib.load('dt_model_cv3.pkl')
print(model)
print(model.feature_importances_)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
[0.24344021 0.2098342  0.03705532 0.03064513 0.14106092 0.11664908
 0.08211952 0.13919563]


In [2]:
import pandas as pd
df = pd.read_csv('../ML-for-stock-port-management/stock_test.csv')
# X_test = df[['Money', 'Money5', 'SMA5', 'Last', 'MACD', 'MACD10', 'RSI']]
# X_test = df[['Last', 'ChangePercent', 'Money']]
X_test = df[['Money', 'Money5', 'SMA5', 'Last', 'MACD', 'MACD10', 'RSI', 'ChangePercent']]
# X_test = df[['Money', 'Money5', 'MACD', 'MACD10', 'ChangePercent']]
y_test = df['Status']

# X_test.columns = ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6']
# X_test.columns = ['f0', 'f1', 'f2']
# X_test.columns = ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6','f7']
# X_test.columns = ['f0', 'f1', 'f2', 'f3', 'f4']

# print(X_test.shape)
# print(y_test.shape)

In [3]:
y_pred = model.predict(X_test)
y_pred.size
# import collections
# collections.Counter(y_pred)

93478

In [7]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred,average='macro')
cm = confusion_matrix(y_test, y_pred)
print('Accuracy: ' + str(accuracy))
print('Precision: ' + str(precision))
print('Recall: ' + str(recall))
print('Confusion Matrix:')
print(cm)

Accuracy: 0.9333212092684909
Precision: 0.759678482969622
Recall: 0.7635707044539751
Confusion Matrix:
[[ 3240  1850    80]
 [ 1734 81106  1329]
 [   73  1167  2899]]


# Save to db format

In [4]:
import pandas as pd

In [5]:
colName = ['date', 'symbol', 'open', 'high', 'low', 'close', 'percentChange', 'volume', 'money', 'macd', 'rsi', 'status']
df = pd.DataFrame(columns = colName)
df.to_csv('trade-db.csv', index=None)
df = pd.read_csv('trade-db.csv')
df

,date,symbol,open,high,low,close,percentChange,volume,money,macd,rsi,status


In [6]:
trade = pd.read_csv('stock_test.csv')
trade.head()

,Date,Symbol,Open,High,Low,Last,ChangePercent,Volumn,Money,Net5D,...,SMA12,SMA26,MACD,MACD5,MACD10,Vol5,RSI,RSI5,Money5,Status
0,8/16/2019,2S,2.40,2.42,2.36,2.40,0.00,1645800.0,3916890.0,13034280.0,...,2.255000,2.260769,0.006225,0.002222,0.007067,986618.6,28.571429,31.428571,2360512.0,0.0
1,8/19/2019,2S,2.44,2.50,2.44,2.48,3.33,1181982.0,2925840.0,15728340.0,...,2.271667,2.269231,0.018875,0.005695,0.007446,1188635.0,28.571429,31.428571,2871270.0,1.0
2,8/20/2019,2S,2.48,2.48,2.40,2.42,-2.42,387100.0,944070.0,15825270.0,...,2.283333,2.275385,0.018262,0.010313,0.007816,1244735.0,28.571429,31.428571,3013728.0,0.0
3,8/21/2019,2S,2.44,2.44,2.40,2.42,0.00,350024.0,845060.0,10235630.0,...,2.301667,2.281538,0.028077,0.016148,0.009309,1238879.8,28.571429,30.000000,3013312.0,0.0
4,8/22/2019,2S,2.40,2.40,2.38,2.40,-0.83,255300.0,611730.0,9243590.0,...,2.316667,2.287692,0.033476,0.020983,0.011483,764041.2,28.571429,28.571429,1848718.0,0.0


In [7]:
oldColName = ['Date', 'Symbol', 'Open', 'High', 'Low', 'Last', 'ChangePercent', 'Volumn', 'Money', 'MACD', 'RSI']
trade = trade.loc[:, oldColName]
trade.to_csv('trade-db.csv', mode='a', index=None, header=False)

In [8]:
# save prediction to csv
trade = pd.read_csv('trade-db.csv')
for i in range(len(trade)):
    trade.loc[i,'status'] = y_pred[i]
trade


,date,symbol,open,high,low,close,percentChange,volume,money,macd,rsi,status
0,8/16/2019,2S,2.40,2.42,2.36,2.40,0.00,1645800.0,3916890.0,0.006225,28.571429,0.0
1,8/19/2019,2S,2.44,2.50,2.44,2.48,3.33,1181982.0,2925840.0,0.018875,28.571429,0.0
2,8/20/2019,2S,2.48,2.48,2.40,2.42,-2.42,387100.0,944070.0,0.018262,28.571429,0.0
3,8/21/2019,2S,2.44,2.44,2.40,2.42,0.00,350024.0,845060.0,0.028077,28.571429,0.0
4,8/22/2019,2S,2.40,2.40,2.38,2.40,-0.83,255300.0,611730.0,0.033476,28.571429,0.0
5,8/23/2019,2S,2.40,2.40,2.36,2.36,-1.67,266600.0,635950.0,0.035883,28.571429,0.0
6,8/26/2019,2S,2.40,2.40,2.30,2.32,-1.69,2005006.0,4652070.0,0.036895,21.428571,0.0
7,8/27/2019,2S,2.34,2.38,2.32,2.34,0.86,316002.0,739900.0,0.046211,28.571429,0.0
8,8/28/2019,2S,2.28,2.32,2.28,2.32,-0.85,386001.0,887670.0,0.051638,28.571429,0.0
9,8/29/2019,2S,2.30,2.44,2.30,2.32,0.00,1709652.0,4038320.0,0.058661,28.571429,0.0


In [9]:
trade.to_csv('trade-db.csv', index=None)